In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import importlib
import yaml, itertools
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import my_utils

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio

import pandas as pd
import random

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import os
import wandb

import glob
import numpy as np
import utils_data_10Fold
import utils_sampler
import copy

from sklearn.metrics import balanced_accuracy_score, confusion_matrix

In [3]:
def make_color_list(emo_list):
    color = {'neu':'slategrey', 'ang':'crimson', 'hap':'gold', 
             'sad':'darkblue',  'dis':'plum', 
             'sur':'steelblue', 'fea':'olivedrab'}
    
    color_list = []
    for e in emo_list:
        c = color[e[:3]]
        color_list.append(c)
        
    return color_list

def make_center_color(color_list):
    color = {'slategrey':'black', 'crimson':'orangered', 'gold':'yellow', 
             'darkblue':'blue',  'plum':'purple', 
             'steelblue':'dodgerblue', 'olivedrab':'darkolivegreen'}
    
    center_list = []
    for e in color_list:
        c = color[e]
        center_list.append(c)
        
    return center_list

In [4]:
class IEMOCAP_Dataset(Dataset):
    def __init__(self, data, hparams):
        
        self.hparams = hparams
        self.csv = data
        self.output_class = hparams['use_class']
        self.vad = hparams['vad']
        self.sr = 16000

        self.aranged_id_num = list(range(1,11))
        self.aranged_id_num.remove(2*hparams['fold_num']-1)
        self.aranged_id_num.remove(2*hparams['fold_num'])
    
    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):

        data = self.csv.iloc[idx]
        ids = data['session']
        ans = self.output_class.index(data['emotion'])

        try:
            pid = self.aranged_id_num.index(data['id_num'])
        except:
            if data['id_num'] % 2 == 0:
                pid = 8
            else:
                pid = 9

        if self.vad:
            wav_path = '/media/ubuntu/SSD2/Dataset/IEMOCAP_VAD/Session'+str(data['fold'])\
                        +'/' + '_'.join(ids.split('_')[:-1]) + '/' + ids +'.wav'
        else:
            wav_path = '/media/ubuntu/SSD2/Dataset/IEMOCAP_full_release/Session'+str(data['fold'])\
                         +'/sentences/wav/' + '_'.join(ids.split('_')[:-1]) + '/' + ids +'.wav'
            if os.getcwd().split('/')[2] == 'cvnar2':
                wav_path = wav_path.replace('/media/ubuntu/SSD2/Dataset', '/home/cvnar2/Desktop/ssd')


        wav, sr = torchaudio.load(wav_path, normalize=True)
        if self.hparams['repeat_3_sec']:
            if wav.shape[-1] / self.sr < 3.0:
                n_repeat = int(3 // (wav.shape[-1] / self.sr))
                wav = wav.repeat((1, n_repeat))
                
        if (data['sec'] > self.hparams['max_sec']):
            max_len = int(16000 * self.hparams['max_sec'])
            offset = random.randint(0, wav.shape[1] - max_len - 1)
            wav = wav[:, offset:offset+max_len]

        inputs = wav.transpose(0, 1).squeeze(1)
        
        return inputs, ans, pid

In [7]:
main_path = '/home/ubuntu/Dropbox/23_for_revision/with_emotion/Vox_IEMO/only_emo_hs'#'Vox_IEMO(3)/main_final2_load_emo_beta/all_finetune_beta(05)'
exp_list = os.listdir(main_path)
exp_list.sort()

bz = 1
if os.path.isdir(main_path + '/plot'):
    pass
else:
    os.mkdir(main_path + '/plot')

In [8]:
exp_list

['1_230215_2208',
 '2_230215_2347',
 '3_230216_0117',
 '4_230216_0258',
 '5_230216_0425',
 'replot']

# 1초 skipper

In [ ]:
import math

class IEMOCAP_Test_Dataset(IEMOCAP_Dataset):
    def __init__(self, data, hparams):
        super(IEMOCAP_Test_Dataset, self).__init__(data, hparams)

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):

        data = self.csv.iloc[idx]
        ids = data['session']
        ans = self.output_class.index(data['emotion'])

        try:
            pid = self.aranged_id_num.index(data['id_num'])
        except:
            if data['id_num'] % 2 == 0:
                pid = 8
            else:
                pid = 9

        if self.vad:
            wav_path = '/media/ubuntu/SSD2/Dataset/IEMOCAP_VAD/Session'+str(data['fold'])\
                        +'/' + '_'.join(ids.split('_')[:-1]) + '/' + ids +'.wav'
        else:
            wav_path = '/media/ubuntu/SSD2/Dataset/IEMOCAP_full_release/Session'+str(data['fold'])\
                         +'/sentences/wav/' + '_'.join(ids.split('_')[:-1]) + '/' + ids +'.wav'
            if os.getcwd().split('/')[2] == 'cvnar2':
                wav_path = wav_path.replace('/media/ubuntu/SSD2/Dataset', '/home/cvnar2/Desktop/ssd')


        wav, sr = torchaudio.load(wav_path, normalize=True)
        if wav.shape[-1] < int(self.sr * self.hparams['max_sec']):
            n = 1
            inputs = wav.transpose(0, 1).squeeze(1)

        elif wav.shape[-1] > int(self.sr * self.hparams['max_sec']):
            inputs = wav.squeeze(0).unfold(0, int(self.sr*self.hparams['max_sec']), self.sr)
            n = len(inputs)

        else:
            pass

        ans = [ans] * n
        pid = [pid] * n
        
        return inputs, ans, pid

In [ ]:
def get_pred_acc(output, answer, balanced_accuracy):

    pred = np.array(output).reshape(-1,1)
    answ = np.array(answer).reshape(-1,1)

    cm = confusion_matrix(answ, pred)
    if balanced_accuracy:
        acc = balanced_accuracy_score(answ, pred)
    else:
        acc = (np.eye(len(cm)) * cm).sum() / cm.sum()

    return acc

def eval_network(model, data_loader, device):

    model.eval()

    i = 0
    emo_answ_dict = []
    emo_pred_dict = []

    for inputs, ans, pid in tqdm(data_loader):
        inputs = inputs.to(device=device)
        if len(inputs.shape) >= 3:
            inputs = inputs.squeeze(0)
        
        ans = torch.Tensor(ans).to(device=device)
        pid = torch.Tensor(pid).to(device=device)

        with torch.no_grad():
            
            out, feat, feat_list = model.get_feat(inputs)

            p = out.clone().detach().cpu().mean(dim=0).argmax(dim=-1).reshape(-1).numpy()[0]
            a = ans[0].clone().detach().cpu().reshape(-1).numpy()[0]

            emo_answ_dict.append(a)
            emo_pred_dict.append(p)

        i += 1

    acc_dict = {'id_UA' :[], 'id_WA':[],
                'emo_UA':[], 'emo_WA':[]}
    
    emo_answ_dict = np.array(emo_answ_dict)
    emo_pred_dict = np.array(emo_pred_dict)

    acc_dict['emo_UA'] = get_pred_acc(emo_pred_dict, emo_answ_dict, balanced_accuracy=True)
    acc_dict['emo_WA'] = get_pred_acc(emo_pred_dict, emo_answ_dict, balanced_accuracy=False)

    #v_acc = acc_dict['emo_UA'] 
    #print(v_acc)
    
    return acc_dict, [emo_answ_dict, emo_pred_dict]

In [ ]:
mode = 'best'
for exp in exp_list:
    if exp == 'plot':
        continue
    print(exp)

    torch.cuda.empty_cache()

    lib_path = glob.glob(main_path+'/'+exp+'/*main*.py')[0][:-3].replace('/', '.')
    saved_main = importlib.import_module(lib_path)

    with open(main_path+'/'+exp+"/hparams.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

    seed = hparams['seed']
    my_utils.set_seed(seed)

    net = saved_main.Emotion_Network(hparams)
    if hparams['id_net_freeze'] != 'freeze':
        net.id_net.hs.fc = nn.Parameter(torch.Tensor(8, hparams['fin_channel']))

    weight = torch.load(main_path+'/'+exp+"/"+mode+"_model.pt")
    missing_keys = net.load_state_dict(weight['model_state_dict'], strict=True)
    print(missing_keys)

    net = net.cuda()
    net.eval()

    emo_list = hparams['use_class']
    
    trainset, validset, testset = utils_data_10Fold.real_5Fold(hparams)
    
    test_set = IEMOCAP_Test_Dataset(testset, hparams)
    testloader = DataLoader(test_set, shuffle=True, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)

    id = '_'.join([hparams['emo_hs_linear'], exp])
    wandb.init(project=hparams['pj_name'], name=id, id=id, resume=True) #, config = hparams)
    # acc_dict, [emo_answ_dict, emo_pred_dict]
    test_acc_dict, [test_emo_answ_dict, test_emo_pred_dict] = eval_network(net, testloader, 'cuda')

    wandb.run.summary["test_"+mode+"_emo_WA"] = test_acc_dict['emo_WA']
    wandb.run.summary["test_"+mode+"_emo_UA"] = test_acc_dict['emo_UA']
    t2 = test_acc_dict['emo_UA']

    fin_print = f'[emo acc: {t2:.3f}'
    print(fin_print)

    wandb.log({"test_"+mode+"_cm" : wandb.plot.confusion_matrix(
                y_true=test_emo_answ_dict, preds=test_emo_pred_dict,
                class_names=emo_list)})

    wandb.finish()

# plot T-sne

In [9]:
## emo: plot id: label 
for exp in exp_list:
    if exp == 'plot':
        continue
    print(exp)

    torch.cuda.empty_cache()

    lib_path = glob.glob(main_path+'/'+exp+'/*main*.py')[0][:-3].replace('/', '.')
    saved_main = importlib.import_module(lib_path)

    with open(main_path+'/'+exp+"/hparams.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

    seed = hparams['seed']
    my_utils.set_seed(seed)

    print(hparams['repeat_3_sec'])

    net = saved_main.Emotion_Network(hparams)
    if hparams['id_net_freeze'] != 'freeze':
        net.id_net.hs.fc = nn.Parameter(torch.Tensor(8, hparams['fin_channel']))

    net.id_filter.data = nn.Parameter(torch.Tensor(1, hparams['pool_head']+1, hparams['fin_channel']))

    weight = torch.load(main_path+'/'+exp+"/best_model.pt")
    missing_keys = net.load_state_dict(weight['model_state_dict'], strict=True)
    print(missing_keys)

    net = net.cuda()
    net.eval()
    
    trainset, validset, testset = utils_data_10Fold.real_5Fold(hparams)
    
    test_set = IEMOCAP_Dataset(testset, hparams)
    testloader = DataLoader(test_set, shuffle=True, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)

    valid_set = IEMOCAP_Dataset(validset, hparams)
    validloader = DataLoader(valid_set, shuffle=True, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)

    trainset = IEMOCAP_Dataset(trainset, hparams)
    trainloader = DataLoader(trainset, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)

    # GE FEATURES
    full_inp = []
    emo_feat_list = []
    id_feat_list = []
    feat_list = []
    new_feat_list = []

    id_list = []
    ans_list = []
    loader_list = []

    emo_list = hparams['use_class']
    color_list = make_color_list(emo_list)

    for idx, iemo_dataloader in enumerate([trainloader, validloader, testloader]):

        if idx == 0:
            name = "train"
        elif idx == 1:
            name = "valid"
        else:
            name = "test"

        for wav, ans, pid in tqdm(iemo_dataloader):
            wav = wav.cuda()

            with torch.no_grad():
                out, feat, feat_list = net.get_feat(wav)

            emo_feat_list.extend(feat.detach().cpu().numpy())
            id_feat_list.extend(feat_list[0].detach().cpu().numpy())
            #feat_list.extend(feat.detach().cpu().numpy())
            #new_feat_list.extend(new_feat.detach().cpu().numpy())

            id_list.append(int(pid[0][0].item()))
            ans_list.append(int(ans[0][0].item()))
            loader_list.append(name)

    id_list = np.array(id_list)
    ans_list = np.array(ans_list)
    loader_list = np.array(loader_list)

    emo_feat_list = np.array(emo_feat_list)
    id_feat_list = np.array(id_feat_list)
    #feat_list = np.array(feat_list)
    #new_feat_list = np.array(new_feat_list)
    
    # PLOT
    tsne = TSNE(n_components=2, perplexity=20, init='pca', learning_rate='auto')
    y_tsne = tsne.fit_transform(emo_feat_list)

    # EMOTION
    fig, ax = plt.subplots(2, 3, figsize=(16,10))
    for n_idx, name in enumerate(['train', 'valid', 'test']):
        mask1 = np.where(loader_list==name, True, False)
        i = 0
        for k in list(set(ans_list)):
            mask2 = np.where(ans_list==k, True, False)

            mask = mask1 & mask2

            x = y_tsne[mask][:, 0]
            y = y_tsne[mask][:, 1]
            ax[0][n_idx].scatter(x, y, label=emo_list[i], color=color_list[i], alpha=0.5, s=10)
            i+=1

        ax[0][n_idx].legend()
        ax[0][n_idx].set_title(exp + ' ' + name)

    # ID
    cmap1 = plt.cm.get_cmap('tab10', 10)
    for n_idx, name in enumerate(['train', 'valid', 'test']):
        mask1 = np.where(loader_list==name, True, False)
        i = 0
        for k in list(set(id_list)):
            mask2 = np.where(id_list==k, True, False)

            mask = mask1 & mask2

            x = y_tsne[mask][:, 0]
            y = y_tsne[mask][:, 1]
            ax[1][n_idx].scatter(x, y, label=k, color=cmap1(i), alpha=0.5, s=10)
            i+=1
        
        ax[1][n_idx].legend()
        ax[1][n_idx].set_title(exp + ' ' + name)

    plt.tight_layout()
    plt.savefig(main_path + '/plot/'+exp+'_emo_id_repeat3sec.png')
    plt.show()

1_230215_2208


IndexError: list index out of range

In [ ]:
## emo: plot id: label 
for exp in exp_list:
    if exp == 'plot':
        continue
    print(exp)

    torch.cuda.empty_cache()

    lib_path = glob.glob(main_path+'/'+exp+'/*main*.py')[0][:-3].replace('/', '.')
    saved_main = importlib.import_module(lib_path)

    with open(main_path+'/'+exp+"/hparams.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

    seed = hparams['seed']
    my_utils.set_seed(seed)

    net = saved_main.Emotion_Network(hparams)
    if hparams['id_net_freeze'] != 'freeze':
        net.id_net.hs.fc = nn.Parameter(torch.Tensor(8, hparams['fin_channel']))

    weight = torch.load(main_path+'/'+exp+"/best_model.pt")
    missing_keys = net.load_state_dict(weight['model_state_dict'], strict=True)
    print(missing_keys)

    net = net.cuda()
    net.eval()
    
    trainset, validset, testset = utils_data_10Fold.real_5Fold(hparams)
    
    test_set = IEMOCAP_Dataset(testset, hparams)
    testloader = DataLoader(test_set, shuffle=True, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)

    valid_set = IEMOCAP_Dataset(validset, hparams)
    validloader = DataLoader(valid_set, shuffle=True, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)

    trainset = IEMOCAP_Dataset(trainset, hparams)
    trainloader = DataLoader(trainset, batch_size=bz, collate_fn=saved_main.Pad_Collate, pin_memory=True)

    """id = '_'.join([hparams['emo_hs_linear'], exp])
    wandb.init(project=hparams['pj_name'], name=id, id=id, resume=True) #, config = hparams)
    _, test_acc_dict, [test_emo_answ_dict, test_emo_pred_dict] = saved_main.eval_network(net, testloader, hparams)

    wandb.run.summary["test_emo_WA"] = test_acc_dict['emo_WA']
    wandb.run.summary["test_emo_UA"] = test_acc_dict['emo_UA']
    t2 = test_acc_dict['emo_UA']

    fin_print = f'[emo acc: {t2:.3f}'
    print(fin_print)

    wandb.log({"test_cm" : wandb.plot.confusion_matrix(
                y_true=test_emo_answ_dict, preds=test_emo_pred_dict,
                class_names=emo_list)})

    wandb.finish()"""

    # GE FEATURES
    

    emo_list = hparams['use_class']
    color_list = make_color_list(emo_list)

    fig, ax = plt.subplots(2, 3, figsize=(16,10))
    for idx, iemo_dataloader in enumerate([trainloader, validloader, testloader]):

        full_inp = []
        emo_feat_list = []
        id_feat_list = []
        feat_list = []
        new_feat_list = []

        id_list = []
        ans_list = []
        loader_list = []

        if idx == 0:
            name = "train"
        elif idx == 1:
            name = "valid"
        else:
            name = "test"

        for wav, ans, pid in tqdm(iemo_dataloader):
            wav = wav.cuda()

            with torch.no_grad():
                out, feat, feat_list = net.get_feat(wav)

            emo_feat_list.extend(feat.detach().cpu().numpy())
            id_feat_list.extend(feat_list[0].detach().cpu().numpy())
            #feat_list.extend(feat.detach().cpu().numpy())
            #new_feat_list.extend(new_feat.detach().cpu().numpy())

            id_list.append(int(pid[0][0].item()))
            ans_list.append(int(ans[0][0].item()))
            loader_list.append(name)

        id_list = np.array(id_list)
        ans_list = np.array(ans_list)
        loader_list = np.array(loader_list)

        emo_feat_list = np.array(emo_feat_list)
        id_feat_list = np.array(id_feat_list)
        #feat_list = np.array(feat_list)
        #new_feat_list = np.array(new_feat_list)
        
        # PLOT
        tsne = TSNE(n_components=2, perplexity=20, init='pca', learning_rate='auto')
        y_tsne = tsne.fit_transform(emo_feat_list)

        # EMOTION
        i = 0
        for k in list(set(ans_list)):
            mask = np.where(ans_list==k, True, False)

            x = y_tsne[mask][:, 0]
            y = y_tsne[mask][:, 1]
            ax[0][idx].scatter(x, y, label=emo_list[i], color=color_list[i], alpha=0.5, s=10)
            i+=1

        ax[0][idx].legend()
        ax[0][idx].set_title(exp + ' ' + name)

        # ID
        cmap1 = plt.cm.get_cmap('tab10', 10)
        i = 0
        for k in list(set(id_list)):
            mask = np.where(id_list==k, True, False)

            x = y_tsne[mask][:, 0]
            y = y_tsne[mask][:, 1]
            ax[1][idx].scatter(x, y, label=k, color=cmap1(i), alpha=0.5, s=10)
            i+=1
        
        ax[1][idx].legend()
        ax[1][idx].set_title(exp + ' ' + name)

    plt.tight_layout()
    plt.savefig(main_path + '/plot/'+exp+'_emo_id_each.png')
    plt.show()        